In [10]:
import mysql.connector
from mysql.connector import Error

# Database connection configuration
config = {
    'user': 'root',
    'password': 'Arnav@tu1',
    'host': 'localhost',
    'database': 'YoutubeData',
    'port': 3306
}

try:
    # Establish a connection to the database
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f'Connected to MySQL Server version {db_info}')
        
        # Create a table
        create_table_query = """
            CREATE TABLE videos (
                id INT AUTO_INCREMENT PRIMARY KEY,
                title VARCHAR(255),
                views INT,
                likes INT,
                dislikes INT
            )
        """
        
        cursor = connection.cursor()
        cursor.execute(create_table_query)
        print("Table 'videos' created successfully")
        
except Error as e:
    print(f'Error while connecting to MySQL: {e}')
    
finally:
    # Close the database connection
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print('MySQL connection is closed')


Connected to MySQL Server version 9.0.0
Table 'videos' created successfully
MySQL connection is closed


In [33]:
import mysql.connector
from googleapiclient.discovery import build
from config import API_KEY, DB_USER, DB_PASSWORD, DB_NAME, DB_HOST, DB_PORT

# Function to fetch and store YouTube video data
def fetch_and_store_youtube_data():
    youtube = build('youtube', 'v3', developerKey=API_KEY)
    region_code = 'US'  # Change this as needed

    request = youtube.videos().list(
        part='snippet,statistics,contentDetails',
        chart='mostPopular',
        regionCode=region_code,
        maxResults=50
    )

    response = request.execute()

    # Connect to MySQL database
    conn = mysql.connector.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        database=DB_NAME,
        port=DB_PORT
    )
    cursor = conn.cursor()

    # Insert data into MySQL database
    for item in response['items']:
        video_id = item['id']
        title = item['snippet']['title']
        views = item['statistics'].get('viewCount', 0)
        likes = item['statistics'].get('likeCount', 0)
        comments = item['statistics'].get('commentCount', 0)
        category = item['snippet']['categoryId']
        published_at = item['snippet']['publishedAt']
        duration = item['contentDetails']['duration']

        cursor.execute('''
            INSERT INTO youtube_videos (id, title, views, likes, comments, category, published_at, duration, region_code)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            views = VALUES(views),
            likes = VALUES(likes),
            comments = VALUES(comments),
            category = VALUES(category),
            published_at = VALUES(published_at),
            duration = VALUES(duration),
            region_code = VALUES(region_code)
        ''', (video_id, title, views, likes, comments, category, published_at, duration, region_code))

    # Commit and close connection
    conn.commit()
    cursor.close()
    conn.close()

# Call the function to fetch and store YouTube data
fetch_and_store_youtube_data()


In [34]:
import mysql.connector
from googleapiclient.discovery import build
from config import API_KEY, DB_USER, DB_PASSWORD, DB_NAME, DB_HOST, DB_PORT

def save_categories_to_db(categories):
    conn = mysql.connector.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        database=DB_NAME,
        port=DB_PORT
    )
    cursor = conn.cursor()

    for category_id, title in categories.items():
        cursor.execute(
            "INSERT INTO youtube_categories (id, title) VALUES (%s, %s) ON DUPLICATE KEY UPDATE title = %s",
            (category_id, title, title)
        )

    conn.commit()
    cursor.close()
    conn.close()
def fetch_youtube_categories():
    youtube = build('youtube', 'v3', developerKey=API_KEY)
    request = youtube.videoCategories().list(
        part='snippet',
        regionCode='US'  # Change this as needed
    )

    response = request.execute()
    categories = {}
    for item in response['items']:
        categories[item['id']] = item['snippet']['title']

    return categories

categories = fetch_youtube_categories()
save_categories_to_db(categories)


In [53]:
import mysql.connector
import csv
from config import DB_USER, DB_PASSWORD, DB_NAME, DB_HOST, DB_PORT

# Function to export data from MySQL database to CSV
def export_data_to_csv():
    conn = mysql.connector.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        database=DB_NAME,
        port=DB_PORT
    )
    cursor = conn.cursor()

    # Fetch data from youtube_videos table with category_title
    cursor.execute('''
        SELECT v.id, v.title, v.views, v.likes, v.comments, v.category, c.title AS category_title,
               v.published_at, v.duration, v.region_code
        FROM youtube_videos v
        LEFT JOIN youtube_categories c ON v.category = c.id
    ''')

    # Fetch all rows from the result set
    rows = cursor.fetchall()

    # Specify the path and filename for the CSV file
    csv_filename = "youtube_videos_with_categories.csv"

    # Open the CSV file in write mode
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        # Create a CSV writer object
        csv_writer = csv.writer(csvfile)
        
        # Write headers to the CSV file
        csv_writer.writerow([i[0] for i in cursor.description])
        
        # Write rows to the CSV file
        csv_writer.writerows(rows)

    print(f"Data exported successfully to {csv_filename}.")

    # Close cursor and connection
    cursor.close()
    conn.close()

# Call the function to export data to CSV
export_data_to_csv()


Data exported successfully to youtube_videos_with_categories.csv.


In [54]:
import csv
from datetime import datetime, time

# Function to update CSV file with cleaned data
def update_csv_file(csv_filename):
    updated_rows = []
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        headers = next(csv_reader)  # Read header row
        
        # Find column indices
        published_at_index = headers.index('published_at')

        # Update header names (if needed)
        headers[published_at_index] = 'Date'  # Assuming 'Date' is already correct
        headers.insert(published_at_index + 1, 'Time')

        for row in csv_reader:
            # Split 'published_at' into 'published_date' and 'published_time'
            published_at = row[published_at_index]
            published_date = datetime.fromisoformat(published_at).date()
            published_time = datetime.fromisoformat(published_at).time()

            row[published_at_index] = str(published_date)
            row.insert(published_at_index + 1, str(published_time))

            # Simplify 'duration' format if it exists
            duration_index = headers.index('duration')
            if len(row) > duration_index:
                duration = row[duration_index]
                if isinstance(duration, time):
                    duration = duration.isoformat()

                # Check if 'duration' starts with 'PT'
                if duration.startswith('PT'):
                    simplified_duration = ''
                    if 'H' in duration:
                        simplified_duration += f"{duration.split('H')[0][2:]}H"
                    if 'M' in duration:
                        simplified_duration += f"{duration.split('M')[0][-2:]}M"
                    if 'S' in duration:
                        simplified_duration += f"{duration.split('S')[0][-2:]}S"

                    row[duration_index] = simplified_duration
                # Otherwise, leave the duration as is

            updated_rows.append(row)

    # Write updated data back to the CSV file
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(headers)  # Write header row with updated names
        csv_writer.writerows(updated_rows)  # Write updated rows

    print(f"CSV file '{csv_filename}' updated successfully.")

# Example usage:
csv_filename = 'youtube_videos_with_categories.csv'
update_csv_file(csv_filename)


CSV file 'youtube_videos_with_categories.csv' updated successfully.
